In [1]:
from env import host, user, password
import acquire
import prepare
import wrangle

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# Acquire

In [3]:
def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [4]:
def get_zillow():
    '''
    This function reads in the Zillow data from the Codeup db
    with properties_2017, predictions_2017 and propertylandusetype tables joined
    returns: a pandas DataFrame 
    '''
    
    zp_query = '''
    SELECT calculatedfinishedsquarefeet, bedroomcnt, bathroomcnt, taxvaluedollarcnt, taxamount, yearbuilt, fips, properties_2017.parcelid
    FROM properties_2017
    JOIN predictions_2017 ON properties_2017.parcelid = predictions_2017.parcelid
    JOIN propertylandusetype ON properties_2017.propertylandusetypeid= propertylandusetype.propertylandusetypeid
    WHERE predictions_2017.transactiondate BETWEEN '2017-05-01' AND '2017-08-31' AND properties_2017.propertylandusetypeid IN (31, 46, 47, 260, 261, 262, 263, 264, 265, 268, 273, 274, 275, 276, 279);
    '''
    return pd.read_sql(zp_query, get_connection('zillow'))

In [5]:
df = get_zillow()

# Prepare

In [6]:
df.head()

,calculatedfinishedsquarefeet,bedroomcnt,bathroomcnt,taxvaluedollarcnt,taxamount,yearbuilt,fips,parcelid
0,1316.0,3.0,2.0,205123.0,2627.48,1923.0,6037.0,11721753
1,1458.0,3.0,2.0,136104.0,2319.90,1970.0,6037.0,11289917
2,1421.0,2.0,1.0,35606.0,543.69,1911.0,6037.0,11705026
3,2541.0,4.0,3.0,880456.0,9819.72,2003.0,6059.0,14269464
4,1491.0,3.0,2.0,107110.0,1399.27,1955.0,6037.0,11446756


In [7]:
#calculate tax rate
df['tax_rate'] =df['taxamount']/  df['taxvaluedollarcnt']

In [8]:
#28,418 entries
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28418 entries, 0 to 28417
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   calculatedfinishedsquarefeet  28345 non-null  float64
 1   bedroomcnt                    28418 non-null  float64
 2   bathroomcnt                   28418 non-null  float64
 3   taxvaluedollarcnt             28417 non-null  float64
 4   taxamount                     28417 non-null  float64
 5   yearbuilt                     28322 non-null  float64
 6   fips                          28418 non-null  float64
 7   parcelid                      28418 non-null  int64  
 8   tax_rate                      28416 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 2.0 MB


In [9]:
#change datatype from float to integer
#df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].astype(int)

In [ ]:
#check for duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#28,385 entries (removed 33 duplicates)
df.info()

In [ ]:
# replace nan values
df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
#drop na values (100 na values)
df.dropna()

In [ ]:
df.nunique()

In [ ]:
sns.boxplot(data=df.bedroomcnt)
plt.title('Bedroom Count')
plt.show()

#outliers can be seen here

In [ ]:
df.bedroomcnt.describe()

In [ ]:
#finding IQR of bedrooms to find outliers
q1_bed = df['bedroomcnt'].quantile(0.25)
q3_bed = df['bedroomcnt'].quantile(0.75)
iqr_bed = q3_bed - q1_bed
iqr_bed, q3_bed, q1_bed

In [18]:
lowerbound_bed = q1_bed - (1.5 * iqr_bed)
upperbound_bed = q3_bed + (1.5 * iqr_bed)
print(f'Lower bounds for bedroom count is: {lowerbound_bed}')
print(f'Upprt bounds for bedroom count is: {upperbound_bed}')

NameError: name 'q1_bed' is not defined

In [ ]:
sns.boxplot(data=df.bathroomcnt)
plt.title('Bathroom Count')
plt.show()

In [ ]:
df.bathroomcnt.describe()

In [14]:
#finding IQR of bathrooms to find outliers
q1_bath = df['bathroomcnt'].quantile(0.25)
q3_bath = df['bathroomcnt'].quantile(0.75)
iqr_bath = q3_bath - q1_bath

iqr_bath, q3_bath, q1_bath

(1.0, 3.0, 2.0)

In [17]:
lowerbound_bath = q1_bath - (1.5 * iqr_bath)
upperbound_bath = q3_bath + (1.5 * iqr_bath)
lowerbound_bath, upperbound_bath
print(f'Lower bounds for bathroom count is: {lowerbound_bath}')
print(f'Upper bounds for bathroom count is: {upperbound_bath}')

Lower bounds for bathroom count is: 0.5
Upprt bounds for bathroom count is: 4.5


In [ ]:
#taxvalue distribution
sns.histplot(data=df.taxvaluedollarcnt)
plt.title('Distribution of Tax Value')
plt.show()

In [ ]:
df.taxvaluedollarcnt.describe()

In [ ]:
#finding IQR of Tax Value to find outliers
q1_tax = df['taxvaluedollarcnt'].quantile(0.25)
q3_tax = df['taxvaluedollarcnt'].quantile(0.75)
iqr_tax = q3_tax- q1_tax
iqr_tax, q3_tax, q1_tax

In [ ]:
lowerbound_tax = q1_tax - (1.5 * iqr_tax)
upperbound_tax = q3_tax + (1.5 * iqr_tax)
print(f'Lower bounds for Assessed Value count is: {lowerbound_tax}')
print(f'Upper bounds for Assessed Value is: {upperbound_tax}')

In [ ]:
#sqft distribution
sns.histplot(data=df.calculatedfinishedsquarefeet)
plt.title('Distribution of Sqft')
plt.show()

In [ ]:
df.calculatedfinishedsquarefeet.describe()

In [ ]:
#finding IQR of Tax Value to find outliers
q1_sqft = df['calculatedfinishedsquarefeet'].quantile(0.25)
q3_sqft = df['calculatedfinishedsquarefeet'].quantile(0.75)
iqr_sqft = q3_sqft - q1_sqft
iqr_sqft, q3_sqft, q1_sqft

In [ ]:
lowerbound_sqrt = q1_sqft - (1.5 * iqr_sqft)
upperbound_sqrt = q3_sqft + (1.5 * iqr_sqft)
print(f'Lower bounds for Square Feet  is: {lowerbound_sqft}')
print(f'Upper bounds for Square Feet is: {upperbound_sqft}')

In [ ]:
#split 


In [ ]:
#take a look at X_train


In [ ]:
#scale data


In [ ]:
#now take a look at X_train after being scaled (minmaxscaler)


# Explore

In [ ]:
y_train.describe()

In [ ]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [ ]:
y_train.head()